The memory module is the primary way pyrit keeps track of requests and responses to targets. The schema is found in `memory_models.py` and can be programatically viewed as follows


In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

from pyrit.memory import DuckDBMemory

memory = DuckDBMemory()

memory.print_schema()

C:\Users\Roman\.conda\envs\pyrit-python311-clean\Lib\site-packages\duckdb_engine\__init__.py:565: SAWarning: Did not recognize type 'list' of column 'embedding'
  columns = self._get_columns_info(rows, domains, enums, schema)  # type: ignore[attr-defined]


C:\Users\Roman\.conda\envs\pyrit-python311-clean\Lib\site-packages\duckdb_engine\__init__.py:180: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


Schema for EmbeddingData:
  Column id (UUID)
  Column embedding (NULL)
  Column embedding_type_name (VARCHAR)
Schema for PromptMemoryEntries:
  Column id (UUID)
  Column role (VARCHAR)
  Column conversation_id (VARCHAR)
  Column sequence (INTEGER)
  Column timestamp (TIMESTAMP)
  Column labels (VARCHAR)
  Column prompt_metadata (VARCHAR)
  Column converter_identifiers (VARCHAR)
  Column prompt_target_identifier (VARCHAR)
  Column orchestrator_identifier (VARCHAR)
  Column response_error (VARCHAR)
  Column original_value_data_type (VARCHAR)
  Column original_value (VARCHAR)
  Column original_value_sha256 (VARCHAR)
  Column converted_value_data_type (VARCHAR)
  Column converted_value (VARCHAR)
  Column converted_value_sha256 (VARCHAR)
Schema for ScoreEntries:
  Column id (UUID)
  Column score_value (VARCHAR)
  Column score_value_description (VARCHAR)
  Column score_type (VARCHAR)
  Column score_category (VARCHAR)
  Column score_rationale (VARCHAR)
  Column score_metadata (VARCHAR)
  Colu


The `pyrit.memory` module provides functionality to keep track of the conversation history. In a nutshell, this can be used as follows:

In [2]:


from uuid import uuid4
from pyrit.models import PromptRequestPiece, PromptRequestResponse

conversation_id = str(uuid4())

message_list = [
    PromptRequestPiece(
        role="user", original_value="Hi, chat bot! This is my initial prompt.", conversation_id=conversation_id
    ),
    PromptRequestPiece(
        role="assistant", original_value="Nice to meet you! This is my response.", conversation_id=conversation_id
    ),
    PromptRequestPiece(
        role="user",
        original_value="Wonderful! This is my second prompt to the chat bot!",
        conversation_id=conversation_id,
    ),
]

memory.add_request_response_to_memory(request=PromptRequestResponse([message_list[0]]))
memory.add_request_response_to_memory(request=PromptRequestResponse([message_list[1]]))
memory.add_request_response_to_memory(request=PromptRequestResponse([message_list[2]]))


entries = memory.get_chat_messages_with_conversation_id(conversation_id=conversation_id)

for entry in entries:
    print(entry)

role='user' content='Hi, chat bot! This is my initial prompt.' name=None tool_calls=None tool_call_id=None
role='assistant' content='Nice to meet you! This is my response.' name=None tool_calls=None tool_call_id=None
role='user' content='Wonderful! This is my second prompt to the chat bot!' name=None tool_calls=None tool_call_id=None


In [3]:

# update based on conversation_id
update_fileds = {"converted_prompt_text": "this is updated field"}
memory.update_entries_by_conversation_id(conversation_id=conversation_id, update_fields=update_fileds)


prompt_entries = memory.get_conversation(conversation_id=conversation_id)

for prompt_entry in prompt_entries:
    print(prompt_entry)

: user: Hi, chat bot! This is my initial prompt.
: assistant: Nice to meet you! This is my response.
: user: Wonderful! This is my second prompt to the chat bot!


In [4]:
# Cleanup memory resources
memory.dispose_engine()